In [5]:
import cv2 as cv 
import numpy as np
from matplotlib import pyplot as plt

#Numpy 中的算法、
#roi 是我们要选的图像中的目标区域
roi = cv.imread("../Images/img1_1.jpg")
roi_hsv = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
#目标对象
img1 = cv.imread("../Images/img1.jpg")
img1_hsv = cv.cvtColor(img1, cv.COLOR_BGR2HSV)
#查找直方图
M = cv.calcHist([roi_hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
I = cv.calcHist([img1_hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])

In [9]:

#求出 R= M/I
R = M / (I + 0.000001)
h, s, v = cv.split(img1_hsv)
B = R[h.ravel(),s.ravel()]
B = np.minimum(B,1)
B = B.reshape(img1_hsv.shape[:2])
disc = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))
cv.filter2D(B,-1,disc,B)
B = np.uint8(B)
cv.normalize(B,B,0,255,cv.NORM_MINMAX)
ret,thresh = cv.threshold(B,50,255,0) 

cv.imshow("target", thresh)
cv.waitKey(0)
cv.destroyAllWindows()




In [10]:
#OpenCV的反投影

# 直方图归一化并利用反传算法
cv.normalize(M,M,0,255,cv.NORM_MINMAX)
dst = cv.calcBackProject([img1_hsv],[0,1],M,[0,180,0,256],1)
# 用圆盘进行卷积
disc = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))
cv.filter2D(dst,-1,disc,dst)
# 应用阈值作与操作
ret,thresh = cv.threshold(dst,50,255,0)
thresh = cv.merge((thresh,thresh,thresh))
res = cv.bitwise_and(img1,thresh)
res = np.vstack((img1,thresh,res))

cv.imshow("target", thresh)
cv.waitKey(0)
cv.destroyAllWindows()